# First Notebook to try out Lightning

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
from glob import glob
import pickle
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
import torchmetrics
from pytorch_lightning.loggers import TensorBoardLogger
from monai.losses import DiceLoss
import albumentations as A
from copy import deepcopy
from segmentation_models_pytorch.losses import SoftBCEWithLogitsLoss
from pytorch_lightning.callbacks import Callback
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
# paths and hyperparameters
name = 'Final_2DCNN'
folder = '/home/olli/Projects/Kaggle/Vesuvius'
folder_data = os.path.join(folder, 'Data', 'Preprocessed_224_20_24')
folder_logs = os.path.join(folder, 'Logs')
depth = 24
size = 224
batch_size = 32
num_workers = 16
epochs=14
learning_rate = 6e-5  # start with this
max_lr = 2e-4  # max value to reach
pct_increase = 0.33  # percentage of epochs to increase the lr before decreasing

# Set up the cross-validation

In [4]:
fold_1_path = os.path.join(folder, 'Data', folder_data, '1')
fold_2_path = os.path.join(folder, 'Data', folder_data, '2')
fold_3_path = os.path.join(folder, 'Data', folder_data, '3')

In [5]:
scans_1 = glob(fold_1_path + '/*.pickle')
scans_2 = glob(fold_2_path + '/*.pickle')
scans_3 = glob(fold_3_path + '/*.pickle')

In [6]:
scans_1[0]

'/home/olli/Projects/Kaggle/Vesuvius/Data/Preprocessed_224_20_24/1/600.pickle'

In [7]:
folds = [scans_1, scans_2, scans_3]

In [8]:
# check number of scans
[len(i) for i in folds]

[1333, 4558, 967]

# Create 3 Train-/Valid-Datasets from the folds

In [9]:
'''Write a function that returns the different train/valid-datasets and -paths.
   Each fold once is the validation data, the rest is for training.'''


def create_train_valid():

    train_folds = ['tr_1', 'tr_2', 'tr_3']
    valid_folds = ['val_1', 'val_2', 'val_3']

    # assign each train/valid dataset the correct folds with i
    for i, train, valid in zip(range(3), train_folds, valid_folds):

        # copy the folds
        flds = deepcopy(folds)

        globals()[valid_folds[i]] = flds.pop(i)  # i'ths index is valid rest is for train
        globals()[train_folds[i]] = flds[0] + flds[1]  # only 0 and 1 left
        
    train_folds = [tr_1, tr_2, tr_3]
    valid_folds = [val_1, val_2, val_3]
    
    return train_folds, valid_folds

In [10]:
train_data, valid_data = create_train_valid()

In [11]:
# check the number of cropped scans in each dataset
[len(i) for i in train_data]

[5525, 2300, 5891]

In [12]:
[len(i) for i in valid_data]

[1333, 4558, 967]

In [13]:
# make sure there is no valid file in the corresponding train dataset
for i in range(3):
    for v in valid_data[i]:
        for t in train_data[i]:
            if v == t:
                print('LEAK')

# Define Augmentations

In [14]:
# start with only a basic augmentation thats safe to use
train_aug = A.Compose([
    #A.Resize(height=224 ,width=224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.82),
    A.ShiftScaleRotate(p=0.47),
    A.OneOf([
            A.GaussNoise(var_limit=[10, 50]),
            A.GaussianBlur(),
            A.MotionBlur(),
            ], p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.32),
    A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3), 
                        mask_fill_value=0, p=0.47),
    A.Normalize(mean=[0] * depth, std=[1] * depth)
])

In [15]:
# use valid aug to normalize the scan
valid_aug = A.Compose([
    #A.Resize(height=224 ,width=224),
    A.Normalize(mean=[0] * depth, std=[1] * depth)
])

# Define the PyTorch Dataset

In [16]:
class Data(Dataset):
    def __init__(self, paths, transform=False):
        self.paths = paths
        self.transform = transform
        
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        path = self.paths[index]
        
        with open(path, 'rb') as f:
            data = pickle.load(f)
            
        X, y = data
        
        if self.transform:
            augmented_data = self.transform(image=X, mask=y)
            
            X = augmented_data['image']
            y = augmented_data['mask']
            
        y = torch.tensor(y / 255.).type(torch.float32)
        
        X = torch.tensor(X).permute(2, 0, 1).type(torch.float32)
        
        return X, y

# Create a Lightning Data Module from that

In [17]:
# create a lightning datamodule class

class DataModule(pl.LightningDataModule):
    
    def __init__(self, train_ds, valid_ds, batch_size=batch_size, num_workers=num_workers):
        
        super().__init__()

        self.batch_size=batch_size
        self.num_workers=num_workers
        
        self.train_ds = train_ds
        self.valid_ds = valid_ds
        
    def setup(self, stage):
        
        self.train_ds = self.train_ds
        self.valid_ds = self.valid_ds
        
    def train_dataloader(self):
        
        train_dl = DataLoader(self.train_ds,
                              batch_size=self.batch_size,
                              num_workers=self.num_workers,
                              shuffle=True,)
                              #pin_memory=True)
        
        return train_dl
        
    def val_dataloader(self):
        
        valid_dl = DataLoader(self.valid_ds,
                              batch_size=self.batch_size,
                              num_workers=self.num_workers,)
                              #pin_memory=True)
        
        return valid_dl

# Define the model

In [18]:
# se_resnext50_32x4d# efficientnet-b0

model = smp.Unet(encoder_name='se_resnext50_32x4d', encoder_weights=None, in_channels=depth, classes=1)

In [19]:
summary(model, input_data=torch.randn(2, depth, size, size))

Layer (type:depth-idx)                             Output Shape              Param #
Unet                                               [2, 1, 224, 224]          --
├─SENetEncoder: 1-1                                [2, 24, 224, 224]         --
│    └─Sequential: 2-1                             --                        --
│    │    └─Conv2d: 3-1                            [2, 64, 112, 112]         75,264
│    │    └─BatchNorm2d: 3-2                       [2, 64, 112, 112]         128
│    │    └─ReLU: 3-3                              [2, 64, 112, 112]         --
│    │    └─MaxPool2d: 3-4                         [2, 64, 56, 56]           --
│    └─Sequential: 2-2                             [2, 256, 56, 56]          --
│    │    └─SEResNeXtBottleneck: 3-5               [2, 256, 56, 56]          71,952
│    │    └─SEResNeXtBottleneck: 3-6               [2, 256, 56, 56]          79,632
│    │    └─SEResNeXtBottleneck: 3-7               [2, 256, 56, 56]          79,632
│    └─Sequential:

In [20]:
# check output shape and make sure that no sigmoid activation is at the last layer
with torch.no_grad():
    print(model(torch.randn(32, depth, size, size)).max())
    print(model(torch.randn(2, depth, size, size)).size())

tensor(5.0035)
torch.Size([2, 1, 224, 224])


In [21]:
# save the initial model weights to use them for each fold
random_weights_path = f'{folder}/Weights/{name}_random.pth'
if not os.path.exists(random_weights_path):
    torch.save(model.state_dict(), random_weights_path)

# Create a custom metric for the dice F 0.5 score

In [22]:
class CustomDice(torchmetrics.Metric):
    def __init__(self, beta=0.5, smooth=1e-5):
        super().__init__()
        
        self.beta = beta
        self.smooth = smooth
        self.add_state("ctp", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("cfp", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("y_true_count", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor, apply_sigmoid=True):
        
        if apply_sigmoid:
            preds = torch.sigmoid(preds)
        preds = preds.view(-1)#.float()
        target = target.view(-1)#.float()

        self.y_true_count += target.sum()
        self.ctp += preds[target == 1].sum()
        self.cfp += preds[target == 0].sum()

    def compute(self):
        c_precision = self.ctp / (self.ctp + self.cfp + self.smooth)
        c_recall = self.ctp / (self.y_true_count + self.smooth)
        beta_squared = self.beta * self.beta
        dice_score = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + self.smooth)

        return dice_score

# Create the Lightning Module

In [23]:
class NN(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.loss_bce = SoftBCEWithLogitsLoss()
        self.loss_dice = DiceLoss(sigmoid=True)  # sigmoid not applied
        #self.loss = SoftBCEWithLogitsLoss(pos_weight=torch.tensor(2))
        #self.loss = DiceLoss(sigmoid=True)
        self.dice = CustomDice()
        
        self.predictions = []  # for holding both to calculate the optimal thresholds
        self.targets = []
        
    def forward(self, x):
        x = self.model(x)
        
        return x
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        
        pred = self.model(X)
        
        # add class dim
        y = y.unsqueeze(1)
        
        #loss = self.loss(pred, y)
        loss_dice = self.loss_dice(pred, y)
        loss_bce = self.loss_bce(pred, y)
        loss = 0.5 * loss_bce + 0.5 * loss_dice
        
        with torch.no_grad():
            dice_score = self.dice(pred, y)
            
        self.log_dict({'train_loss': loss,
                       'train_dice': dice_score},
                       on_step=False,
                       on_epoch=True,
                       prog_bar=True,
                       logger=True)
        
        return {'loss': loss, 'train_dice': dice_score}
        
    def validation_step(self, batch, batch_idx):
        X, y = batch
        
        pred = self.model(X)
        
        # append the pred and y for thresholding
        self.predictions.append(pred.flatten()) # flatten them for calculation
        self.targets.append(y.flatten())
        
        y = y.unsqueeze(1)
        
        #loss = self.loss(pred, y)
        loss_dice = self.loss_dice(pred, y)
        loss_bce = self.loss_bce(pred, y)
        loss = 0.5 * loss_bce + 0.5 * loss_dice
        
        dice_score = self.dice(pred, y)
        
        self.log_dict({'valid_loss': loss,
                       'valid_dice': dice_score},
                        on_step=False,
                        on_epoch=True,
                        prog_bar=True,
                        logger=True)
        
        return {'valid_loss': loss, 'valid_dice': dice_score}
    
    def on_validation_epoch_end(self):
        thresholds = np.arange(start=0.2, stop=0.6, step=0.05)  # thresholds to try
        scores = []  # append the scores here to log them after all done
        
        # use the attributes to calculate the dice score for each threshold
        predictions = torch.cat(self.predictions, dim=0)
        targets = torch.cat(self.targets, dim=0)
        
        # first apply sigmoid to apply correct threshold
        predictions = torch.sigmoid(predictions)
        
        # try all thresholds
        for threshold in thresholds:
            threshold = torch.tensor(threshold)
            
            # make predictions binary based on threshold            
            pred_th = torch.where(predictions > threshold, 1, 0)

            dice_score = self.dice(pred_th, targets, apply_sigmoid=False).cpu()
            print('TH: ', threshold, ' - Dice: ', dice_score)
            scores.append(dice_score)
        
        # get the index of the best dice score
        index_best = torch.argmax(torch.tensor(scores))
        
        best_dice = torch.round(scores[index_best].clone(), decimals=3)
        best_threshold = thresholds[index_best]
        
        print(f'Best Threshold: {best_threshold} - Best Dice Score: {best_dice}')
        
        # Log the best dice valid        
        self.log_dict({'Best_Threshold': torch.tensor(best_threshold), 'Best_Dice': best_dice},
                       on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        # reset after each epoch
        self.predictions = []
        self.targets = []
        
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        # needs params: total steps or steps per epoch and epochs
        total_steps = len(self.trainer.datamodule.train_dataloader()) * epochs
        scheduler = OneCycleLR(optimizer=optim,
                               max_lr=1e-4,
                               epochs=epochs,
                               #steps_per_epoch=10,
                               total_steps = total_steps, #epochs*10,
                               pct_start=pct_increase,)
                               #verbose=True)
        return [optim], [{"scheduler": scheduler, "interval": "step"}]

In [24]:
MODEL = NN(model=model, learning_rate=learning_rate)

# Train / validate 3 models with the random weights

In [25]:
%%time

for i in range(0, 3):
    MODEL.model.load_state_dict(torch.load(random_weights_path))
    train_ds = Data(paths=train_data[i], transform=train_aug)
    valid_ds = Data(paths=valid_data[i], transform=valid_aug)
    dm = DataModule(train_ds=train_ds, valid_ds=valid_ds)
    logger = TensorBoardLogger(folder_logs, name=name, version=f'Fold_{i+1}')
    trainer = pl.Trainer(accelerator='gpu', devices=[0], max_epochs=epochs, precision='16-mixed', logger=logger)
    trainer.fit(MODEL, dm)
    path_save_weights = os.path.join(folder, 'Weights', f'Model_{i + 1}_{name}.pth')
    torch.save(MODEL.model.state_dict(), path_save_weights)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | Unet                  | 34.6 M
1 | loss_bce  | SoftBCEWithLogitsLoss | 0     
2 | loss_dice | DiceLoss              | 0     
3 | dice      | CustomDice            | 0     
----------------------------------------------------
34.6 M    Trainable params
0         Non-trainable params
34.6 M    Total params
138.359   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.3338)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.1197)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.)
Best Threshold: 0.2 - Best Dice Score: 0.33399999141693115


/home/olli/anaconda3/envs/light/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Training: 0it [00:00, ?it/s]

...

TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.4333)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.4526)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.4693)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.4838)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.4958)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.5068)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.5123)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.5127)
Best Threshold: 0.5499999999999999 - Best Dice Score: 0.5130000114440918


Validation: 0it [00:00, ?it/s]

TH:  tensor(0.2000, dtype=torch.float64)  - Dice:  tensor(0.4190)
TH:  tensor(0.2500, dtype=torch.float64)  - Dice:  tensor(0.4382)
TH:  tensor(0.3000, dtype=torch.float64)  - Dice:  tensor(0.4540)
TH:  tensor(0.3500, dtype=torch.float64)  - Dice:  tensor(0.4692)
TH:  tensor(0.4000, dtype=torch.float64)  - Dice:  tensor(0.4824)
TH:  tensor(0.4500, dtype=torch.float64)  - Dice:  tensor(0.4965)
TH:  tensor(0.5000, dtype=torch.float64)  - Dice:  tensor(0.5066)
TH:  tensor(0.5500, dtype=torch.float64)  - Dice:  tensor(0.5111)
Best Threshold: 0.5499999999999999 - Best Dice Score: 0.5109999775886536


`Trainer.fit` stopped: `max_epochs=14` reached.


CPU times: user 13min 45s, sys: 4min, total: 17min 46s
Wall time: 46min 33s
